In [18]:
from datetime import datetime
import matplotlib.pyplot as plt
import meteostat
#https://dev.meteostat.net/python/
from meteostat import units, Stations
import pandas as pd
import numpy as np
from time import time
pd.set_option('display.max_columns', 150)
import geopy.distance
from os import listdir
from os.path import isfile, join

### Import Cloudy Data

In [19]:
cloudy_data = pd.read_csv('DataSets/Weather/zip_lat_lon_wban_cloudiness_data.csv')
cloudy_data['LAT-WBAN'] = cloudy_data['LAT']
cloudy_data['LNG-WBAN'] = cloudy_data['LNG']
cloudy_data['wban-distance-km'] = cloudy_data['wban_distance_km']
def zip_str(row):
    zip_str = str(int(row['ZIP']))
    while len(zip_str) < 5:
        zip_str = '0' + zip_str
    return zip_str
cloudy_data['zip_str'] = cloudy_data.apply(lambda row: zip_str(row), axis =1)
col_to_drop = ['Unnamed: 0', 'Jan CL_zipcode', 'Yrs_zipcode', 'closestWBAN', 
       'Jan PC_zipcode', 'Jan CD_zipcode', 'Feb CL_zipcode', 'Feb PC_zipcode',
       'Feb CD_zipcode', 'Mar CL_zipcode', 'Mar PC_zipcode', 'Mar CD_zipcode',
       'Apr CL_zipcode', 'Apr PC_zipcode', 'Apr CD_zipcode', 'May CL_zipcode',
       'May PC_zipcode', 'May CD_zipcode', 'Jun CL_zipcode', 'Jun PC_zipcode',
       'Jun CD_zipcode', 'Jul CL_zipcode', 'Jul PC_zipcode', 'Jul CD_zipcode',
       'Aug CL_zipcode', 'Aug PC_zipcode', 'Aug CD_zipcode', 'Sept CL_zipcode',
       'Sept PC_zipcode', 'Sept CD_zipcode', 'Oct CL_zipcode',
       'Oct PC_zipcode', 'Oct CD_zipcode', 'Nov CL_zipcode', 'Nov PC_zipcode',
       'Nov CD_zipcode', 'Dec CL_zipcode', 'Dec PC_zipcode', 'Dec CD_zipcode',
       'Ann  CL_zipcode', 'Ann  PC_zipcode', 'Ann  CD_zipcode',
      'State_zipcode', 'City_zipcode', 'LAT_wban', 'LNG_wban', 'LAT', 'LNG',
      'wban_distance_km', 'ZIP']
cloudy_data = cloudy_data.drop(col_to_drop, axis=1)
cloudy_data.columns = [column.split('_')[0] for column in cloudy_data.columns]

In [20]:
for zip_code in cloudy_data['zip']:
    # Check if file already exists in output
    if any([zip_code in file for file in listdir('DataSets/Weather/Daily/')]):
        pass
    else:
        try:
            lat = cloudy_data[cloudy_data['zip'] == zip_code]['LAT'].values[0]
            lng = cloudy_data[cloudy_data['zip'] == zip_code]['LNG'].values[0]

            # wban_zip = ziplatlonwban[ziplatlonwban['zip_str'] == zip_code]
            #Create point for specfied zip_code
            location = meteostat.Point(lat, lng)
            #Set time period
            start = datetime(2019, 1, 1)
            end = datetime(2020, 1, 1)

            start_time = time()

            #Get monthly data
            data = meteostat.Hourly(location, start, end)
            data = data.convert(units.imperial)
            data = data.fetch()

            #Weather condition codes
            # https://dev.meteostat.net/formats.html#weather-condition-codes
            # data['sunny'] = data['coco'] < 3
            # data['rainy'] = (data['coco'] > 6) | (data['prcp'] > 0)

            #Scoping to afternoon data for dwpt calculation
            afternoon = data[(data.index.hour > 11) & (data.index.hour < 19)]
            afternoon_rhum = afternoon.groupby([afternoon.index.month, afternoon.index.day]).mean()['rhum']
            afternoon_rhum.index = afternoon_rhum.index.set_names(['Month', 'Day'])
            afternoon_rhum = afternoon_rhum.reset_index()
            afternoon_rhum_mthly = pd.DataFrame(afternoon_rhum.groupby('Month').mean()['rhum'])
            afternoon_rhum_mthly.columns = ['mthly_afternoon_rhum']

            grouped_by_day = data.groupby([data.index.month, 
                                           data.index.day]).agg({'temp' : ['mean', 'min', 'max', 'count'], 
                                                                            'rhum' : ['mean', 'min', 'max'], 
                                                                            'prcp' : ['sum', 'min', 'count']})
            #                                                                 'rainy' : np.max,
            #                                                                 'sunny' : np.max})

            grouped_by_day.index = grouped_by_day.index.set_names(['Month', 'Day'])
            grouped_by_day = grouped_by_day.reset_index()
            grouped_by_day.columns = ['month', 'day', 'tempmean', 'tempmin', 'tempmax', 'tempcount', 'rhummean', 'rhummin', 'rhummax', 'prcpsum', 'prcpmin', 'prcpcount']#, 'rainymax', 'sunnymax']
            grouped_by_day['over90'] = grouped_by_day['tempmax'] > 90

            grouped_by_month = grouped_by_day.groupby('month').agg({'tempmean' : 'mean',
                                                                   'tempmin' : 'mean',
                                                                   'tempmax' : 'mean',
                                                                   'tempcount' : 'sum',
                                                                   'rhummean' : 'mean',
                                                                   'rhummin' : 'mean',
                                                                   'rhummax' : 'mean',
                                                                   'prcpsum' : 'sum',
                                                                   'prcpmin' : 'mean',
                                                                    'prcpcount' : 'sum',
            #                                                        'rainymax' : 'sum',
            #                                                         'sunnymax' : 'sum',
                                                                    'over90' : 'sum'
                                                                   })

            season_dict = {1:'winter',
                          2: 'winter',
                          3: 'spring',
                          4: 'spring',
                          5: 'spring',
                          6: 'summer',
                          7: 'summer',
                          8: 'summer',
                          9: 'fall',
                          10:'fall',
                          11:'fall',
                          12:'winter'}

            grouped_by_month['season'] = grouped_by_month.index.map(season_dict)

            grouped_by_month = grouped_by_month.merge(afternoon_rhum_mthly, left_index=True, right_index=True, how='left')

            #calculating temp-humidity index (High is good)
            thi_bins = [-1, 75, 85, 95, 105, 1000]
            grouped_by_month['THI'] = grouped_by_month['tempmean'] - 0.55 * (1 - grouped_by_month['mthly_afternoon_rhum']/100) * (grouped_by_month['tempmean'] - 58)
            grouped_by_month['THI_score'] = pd.cut(grouped_by_month['THI'], thi_bins, labels = [len(thi_bins) - 2 - i for i in range(0, len(thi_bins) - 1)]).astype(int)

            #calculating temp_min index (High is good)
            temp_min_bins = [-1, 15, 25, 35, 45, 1000]
            grouped_by_month['tempmin_score'] = pd.cut(grouped_by_month['tempmin'], temp_min_bins, labels = [i for i in range(0, len(temp_min_bins) - 1)]).astype(int)

            #calculating over90 index (High is good)
            over90_bins = [-1, 3, 6, 9, 12, 15, 20, 1000]
            grouped_by_month['over90_score'] = pd.cut(grouped_by_month['over90'], over90_bins, labels = [len(thi_bins) - i for i in range(0, len(over90_bins) - 1)]).astype(int)

            #calculating rainy index (High is good)
            # rainy_bins = [-1, 3, 6, 10, 15, 20, 25, 1000]
            # grouped_by_month['rainy_score'] = pd.cut(grouped_by_month['rainymax'], rainy_bins, labels = [len(rainy_bins) - 2 - i for i in range(0, len(rainy_bins) - 1)]).astype(int)

            #calculating total rain index (High is good)
            # Not sure if we should include this one...
            rainy_sum_bins = [0.00001, 1, 2, 3, 4, 5, 6, 1000]
            grouped_by_month['rainy_sum_score'] = pd.cut(grouped_by_month['prcpsum'], rainy_sum_bins, labels = [len(rainy_sum_bins) - 2 - i for i in range(0, len(rainy_sum_bins) - 1)])
            # Filling months without rain with slightly penalized score as no one likes a complete desert
            grouped_by_month['rainy_sum_score'] = grouped_by_month['rainy_sum_score'].fillna(len(rainy_sum_bins) - 3).astype(int)

            #Adding in cloudy data
            cloud_score_bins = [0, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 70]
            CL_columns = cloudy_data.columns[['CL' in x for x in cloudy_data.columns]]
            PC_columns = cloudy_data.columns[['PC' in x for x in cloudy_data.columns]]
            CD_columns = cloudy_data.columns[['CD' in x for x in cloudy_data.columns]]

            cloudiness_temp = cloudy_data[cloudy_data['zip'] == zip_code][CL_columns].T.reset_index().iloc[:12]
            cloudiness_temp['PC_days'] = cloudy_data[cloudy_data['zip'] == zip_code][PC_columns].T.reset_index().iloc[:12].iloc[:,1:].values
            cloudiness_temp['CD_days'] = cloudy_data[cloudy_data['zip'] == zip_code][CD_columns].T.reset_index().iloc[:12].iloc[:,1:].values
            cloudiness_temp = cloudiness_temp.iloc[:,1:]
            cloudiness_temp.index = [x+1 for x in range(12)]
            cloudiness_temp.columns = ['CL_days', 'PC_days', 'CD_days']
            #Creating cloud score
            cloudiness_temp['CloudScore'] = cloudiness_temp['CL_days'] * 2 + cloudiness_temp['PC_days']
            cloudiness_temp['cloud_score_binned'] = pd.cut(cloudiness_temp['CloudScore'], cloud_score_bins, labels = [i for i in range(0, len(cloud_score_bins) - 1)]).astype(int)

            grouped_by_month = grouped_by_month.merge(cloudiness_temp, how='left', left_index=True, right_index=True)

            # #Adding in sunny data
            # sunny_bins = [0, 20, 30, 40, 50, 60, 70, 80, 90, 101]
            # sunny_temp = sunny_perc[sunny_perc['zip_str'] == zip_code].T.reset_index().iloc[:12]
            # sunny_temp = sunny_temp.iloc[:,1:].astype(int)
            # sunny_temp.index = [x+1 for x in range(12)]
            # sunny_temp.columns = ['Sunny_perc']
            # sunny_temp['sunny_score_binned'] = pd.cut(sunny_temp['Sunny_perc'], sunny_bins, labels = [i for i in range(0, len(sunny_bins) - 1)]).astype(int)

            # grouped_by_month = grouped_by_month.merge(sunny_temp, how='left', left_index=True, right_index=True)

            #Calculating weather index
            thi_weight = 30
            mintemp_weight = 5
            over90_weight = 20
            rainy_sum_weight = 5
            cloud_score_weight = 40
            # sunny_perc_weight = 20
            # rainy_day_weight = 5

            max_thi_score = len(thi_bins) - 2
            max_temp_min_score = len(temp_min_bins) - 2
            max_over_90_score = len(over90_bins) - 2
            max_rainy_sum_score = len(rainy_sum_bins) - 2
            max_cloud_score = len(cloud_score_bins) - 2
            # max_sunny_score = len(sunny_bins) - 2
            # max_rainy_score = len(rainy_bins) - 2

            grouped_by_month['WeatherIndex'] = (grouped_by_month['THI_score'] / max_thi_score * thi_weight) \
                                               + (grouped_by_month['tempmin_score'] / max_temp_min_score * mintemp_weight) \
                                               + (grouped_by_month['over90_score'] / max_over_90_score * over90_weight) \
                                               + (grouped_by_month['rainy_sum_score'] / max_rainy_sum_score * rainy_sum_weight) \
                                               + (grouped_by_month['cloud_score_binned'] / max_cloud_score * cloud_score_weight) \
            #                                    + (grouped_by_month['sunny_score_binned'] / max_sunny_score * sunny_perc_weight)
            #                                    + (grouped_by_month['rainy_score'] / max_rainy_score * rainy_day_weight) \


            #Aggregating by season
            grouped_by_season = grouped_by_month.groupby('season').agg({'tempmean' : 'mean',
                                                                    'tempmin' : 'mean',
                                                                    'tempmax' : 'mean',
                                                                    'tempcount' : 'sum',
                                                                    'rhummean' : 'mean',
                                                                    'rhummin' : 'mean',
                                                                    'rhummax' : 'mean',
                                                                    'prcpsum' : 'sum',
                                                                    'prcpmin' : 'mean',
                                                                    'prcpcount' : 'sum',
            #                                                         'rainymax' : 'sum',
            #                                                         'sunnymax' : 'sum',
                                                                    'over90' : 'sum',
                                                                    'mthly_afternoon_rhum' : 'mean',
                                                                    'THI' : 'mean',
                                                                    'THI_score' : 'mean',
                                                                    'tempmin_score' : 'mean',
                                                                    'over90_score' : 'mean', 
            #                                                         'rainy_score' : 'mean',
                                                                    'rainy_sum_score' : 'mean',
                                                                    'CL_days' : 'sum',
                                                                    'PC_days' : 'sum',
                                                                    'CD_days' : 'sum',
                                                                    'cloud_score_binned' : 'mean',
                                                                    'WeatherIndex' : 'mean'
                                                                   })
            grouped_by_day.to_csv('DataSets/Weather/Daily/{}_2019daily.csv'.format(zip_code))
            grouped_by_month.to_csv('DataSets/Weather/Monthly/{}_2019monthly.csv'.format(zip_code))
            grouped_by_season.to_csv('DataSets/Weather/Seasonal/{}_2019seasonal.csv'.format(zip_code))

            end_time = time()
            print('Output data for zip: {} in {} ms'.format(zip_code, round(end_time - start_time, 2)))
        except:
            print('BREAK! Unable to output data for zip: {}'.format(zip_code))
            pass

BREAK! Unable to output data for zip: 00601
BREAK! Unable to output data for zip: 00602
BREAK! Unable to output data for zip: 00603
BREAK! Unable to output data for zip: 00606
BREAK! Unable to output data for zip: 00610
BREAK! Unable to output data for zip: 00612
BREAK! Unable to output data for zip: 00616
BREAK! Unable to output data for zip: 00617
BREAK! Unable to output data for zip: 00622
BREAK! Unable to output data for zip: 00623
BREAK! Unable to output data for zip: 00624
BREAK! Unable to output data for zip: 00627
BREAK! Unable to output data for zip: 00631
BREAK! Unable to output data for zip: 00637
BREAK! Unable to output data for zip: 00638
BREAK! Unable to output data for zip: 00641
BREAK! Unable to output data for zip: 00647
BREAK! Unable to output data for zip: 00650
BREAK! Unable to output data for zip: 00652
BREAK! Unable to output data for zip: 00653
BREAK! Unable to output data for zip: 00656
BREAK! Unable to output data for zip: 00659
BREAK! Unable to output data for